In [ ]:
from astropy.table import Table, vstack
import glob
import pandas as pd
import numpy as np 
from astropy.time import Time
from astropy.table import Table
from astropy.io import fits
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import time
import random
from scipy.stats import sem
from datetime import datetime, timezone
from matplotlib.patches import Circle
import matplotlib.patches as patches
from matplotlib.patches import RegularPolygon
import numpy as np

In [ ]:
random.uniform(-np.pi, np.pi)

#### LST mirror coordinates

In [ ]:
# Define the path to your .dat file
file_path = '/Users/vdk/Downloads/lst-sim-config-sim-telarray-lst-magic-prod2-v1.4/mirror_CTA-N-LST1_v2019-03-31.dat'

# Lists to hold the extracted data from the first two columns
first_column = []
second_column = []

# Open the file and read the data
with open(file_path, 'r') as file:
    for line in file:
        # Split the line into a list of elements based on whitespace
        columns = line.split()
        # Check if the line has enough columns
        if len(columns) >= 2 and line[0] != '#':
            # Append the data from the first two columns to their respective lists
            first_column.append(float(columns[0]))
            second_column.append(float(columns[1]))

# Output the lists
print("First column:", first_column)
print("Second column:", second_column)


In [ ]:
dat = Table.read('/Users/vdk/muons2024/lapalma_simulations/cta-config/fit_valid/muon_table_run1.fits', format='fits')
sum(dat['parameters_at_limit'])

In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
print(sorted(muon_files))
df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['ring_containment'] > 0.9) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
#df_good_data = df_all_data[(df_all_data['ring_containment'] > 0.999) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
del(df_all_data)
df_good_data

#### Simulation upload

In [ ]:
#listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0784-reflection/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/cscs_simulations/tables_large_lst_muons/*')
listdir3 = glob.glob('/Users/vdk/muons2024/cscs_simulations/muons_zenith0/*')
#listdir4 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0684-reflection/tables/*')
#listdir5 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0884-reflection/tables/*')
listdir7 = glob.glob('/Users/vdk/muons2024/cscs_simulations/narrow_cone/tables/*')
# Muons analyzed using GlobalPeakWindowSum
listdir=listdir2+listdir3+listdir7
#listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/theta_40_az_180/tables/*')
listdir_z0 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/zenith0/tables_muon/*')
listdir_z0_pt2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/zenith0/tables/*')
listdir_z10 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/theta_10.0_az_180/tables/*')
listdir_z20 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/0784-reflection/tables/*')
listdir_z30 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/theta_30_az_180/tables/*')
listdir_z40 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/theta_40_az_180/tables/*')

listdir = listdir_z0 + listdir_z0_pt2 + listdir7 + listdir3 + listdir2

listdir_z0_cta1 = glob.glob("/Users/vdk/muons2024/lapalma_simulations/cta-config/fit_valid/tables/*")
listdir_z0_cta2 = glob.glob("/Users/vdk/muons2024/lapalma_simulations/cta-config/tables_pt2/*")

listdir_z0_cta = glob.glob("/Users/vdk/muons2024/lapalma_simulations/cta-config/tables/*")
listdir_z0_cta_fit_valid = glob.glob('/Users/vdk/muons2024/lapalma_simulations/cta-config/fit_valid/tables/*')

listdir_mir_align00125 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/mirror_allignment/random_00125/tables/*')

listdir_fitbound = glob.glob('/Users/vdk/muons2024/lapalma_simulations/fit_play/boundaries0-2pi/tables/*')
listdir_fit_random = glob.glob('/Users/vdk/muons2024/lapalma_simulations/fit_play/fit_random/tables/*')
listdir=listdir_z0_cta1+listdir_z0_cta2

listdir_fit_random_initial_orig = glob.glob("/Users/vdk/muons2024/lapalma_simulations/fit_play/fit_random_initial_orig/tables/*")
listdir_fit_random_initial_guess_minus01 = glob.glob("/Users/vdk/muons2024/lapalma_simulations/fit_play/fit_random_initial_guess_phi_minus01/tables/*")
listdir_fit_random_initial_guess_pina2 = glob.glob("/Users/vdk/muons2024/lapalma_simulations/fit_play/fit_random_initial_guess_phi_pina2/tables/*")

dat = Table.read('{}'.format(listdir_fit_random_initial_guess_minus01[0]), format='fits')
for muon_file in listdir_fit_random_initial_guess_minus01:
    dat2 = Table.read(muon_file, format='fits')
    dat2['good_ring'] = dat2['good_ring'].astype(bool)
    dat2['is_valid'] = dat2['is_valid'].astype(bool)
    dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)
    dat = vstack([dat, dat2])

min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 
df = dat.to_pandas()
df_good_sim_data = df[(df['muon_efficiency'] < 1) & 
                      (df['ring_containment'] > 0.9) & 
                      (df['size_outside'] < 500) &
                      (df['impact_parameter'] < max_impact) &
                      (df['impact_parameter'] > min_impact)]
#df_good_data = df_good_data[df_good_data['ring_completeness'] > 0.9]
print(np.mean(df_good_sim_data['muon_efficiency']))
df_good_sim_data

In [ ]:
df_good_sim_data

In [ ]:
number = 7
#cos = abs(df_good_data['impact_x_array'][number]) / df_good_data['impact_parameter'][number] 
cosines = df_good_data['impact_x_array'] / df_good_data['impact_parameter'] 
phi = np.rad2deg(np.arccos(cosines))
phi_cut = phi[:1000000]
max(phi_cut)

In [ ]:
df_good_data_cut = copy.deepcopy(df_good_data[:3000000])
condition = (
    np.rad2deg(
        np.arccos(
            np.abs(df_good_data_cut['impact_x_array']) / df_good_data_cut['impact_parameter']
        )
    ) < 59
) | (
    np.rad2deg(
        np.arccos(
            np.abs(df_good_data_cut['impact_x_array']) / df_good_data_cut['impact_parameter']
        )
    ) > 60
)
sum(condition)
len(df_good_data_cut)-sum(condition)
alpha_value = 0.01
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
#circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
#ax.add_patch(circle)
for mirror_x, mirror_y in zip(first_column,second_column):
    circle = Circle((mirror_y*0.01, (mirror_x*0.01-0.269)), 0.75, color='k', fill=False, alpha = 0.3)  # Center at (0.5, 0.5) with radius 0.2
    ax.add_patch(circle)
# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_good_data_cut['impact_x_array'][~condition], df_good_data_cut['impact_y_array'][~condition], alpha = 0.005, s = 1, marker = 'x', c = 'red')
ax.scatter(df_good_data_cut['impact_x_array'][condition], df_good_data_cut['impact_y_array'][condition], alpha = 0.05, s = 1, marker = 'x')

ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of the observed {(upper_limit-low_limit):.0E} muons on the LST mirror, transparency of dots = {alpha_value}")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
df

In [ ]:
cosines_sim = np.arccos(df_good_sim_data['impact_x_array'] / df_good_sim_data['impact_parameter'])

#cosines_sim_cut = np.rad2deg(cosines_sim_cut)
cosines_sim_cut = np.rad2deg(cosines_sim)
cosines_sim_cut

In [ ]:
z,x,c = plt.hist(cosines_sim_cut, bins = 40, density=True)
plt.grid(alpha=0.5)

In [ ]:
len(cosines_sim_cut[(cosines_sim_cut>56)])

In [ ]:
cosines_sim = df_good_sim_data['impact_x_array'][(df_good_sim_data['is_valid']) | (df_good_sim_data['parameters_at_limit'])] / df_good_sim_data['impact_parameter'][(df_good_sim_data['is_valid']) | (df_good_sim_data['parameters_at_limit'])]
cosines_sim_cut = cosines_sim[:1000000]
cosines_sim_cut = np.rad2deg(cosines_sim_cut)
min(cosines_sim_cut)

In [ ]:
z,x,c = plt.hist(cosines_sim_cut, bins = 40, density=True)
plt.grid(alpha=0.5)

In [ ]:
condition = (np.rad2deg(np.arccos((abs(df_good_sim_data['impact_x_array']) / df_good_sim_data['impact_parameter']))) < 56)
without_problems = df_good_sim_data[condition]

In [ ]:
without_problems

### All observed muons

In [ ]:
low_limit = 0
upper_limit = len(df_good_data['impact_parameter'])
alpha_value = 1
fig, ax = plt.subplots(figsize=(9,9))
#circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
#ax.add_patch(circle)


ax.scatter(df_good_data['impact_x_array'][:upper_limit], df_good_data['impact_y_array'][:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of the observed {(upper_limit-low_limit):.3E} muons on the LST mirror")#, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.15, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
ax.set_aspect('equal')
#ax.legend(handles, labels)
#plt.savefig(f'/Users/vdk/muons2024/images/ZenithImpactStudy/ImpactPoints_AllYears_Alpha{alpha_value}.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

### Less number of observed muons

In [ ]:
low_limit = 6000000 #(7*1353641)
upper_limit = 100000000 #(8*1353641)
alpha_value = 0.002
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
#circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
#ax.add_patch(circle)

x = df_good_data['impact_x_array'][df_good_data['ring_completeness'] > 0][low_limit:upper_limit]
y = df_good_data['impact_y_array'][df_good_data['ring_completeness'] > 0][low_limit:upper_limit]
# Add the circle to the Axes


# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(x, y, alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of observed {(upper_limit-low_limit):.0E} the muons on the LST mirror") #, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.3, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
#ax.legend(handles, labels)
plt.savefig(f'/Users/vdk/muons2024/images/ImpactPoints_{(low_limit)}to{(upper_limit)}muons_Alpha{alpha_value}.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

#### Play with different dates

In [ ]:
start_date = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
x_arr = df_good_data['impact_x_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)]
y_arr = df_good_data['impact_y_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)]
print(f"Number of points = {len(x_arr)}")
alpha_value = 0.007
fig, ax = plt.subplots(figsize=(12,12))
ax.set_aspect('equal')

ax.scatter(x_arr, y_arr, alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
#ax.set_title(f"Reconstructed impact points of observed {(upper_limit-low_limit):.0E} the muons on the LST mirror, transparency of dots = 1")

for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.3, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
#plt.savefig(f'/Users/vdk/muons2024/images/TimeImpactStudy/ImpactPoints_{len(x_arr)}muons_2021year_Alpha{alpha_value}.png', dpi=100, format='png', bbox_inches='tight')
plt.xticks(np.arange(-11, 11, 0.5), fontsize = 6)  # Adjust the 0.5 to make it denser or less dense
plt.yticks(np.arange(-11, 11, 0.5)) 
plt.grid(alpha=0.75)
#plt.show()

In [ ]:
xmin, xmax = 6.5, 8
ymin, ymax = -5.5, -4

# Define conditions
condition_x = (x_arr >= xmin) & (x_arr <= xmax)
condition_y = (y_arr >= ymin) & (y_arr <= ymax)

# Combine conditions
combined_condition = condition_x & condition_y

# Filter indices
filtered_indices = x_arr[combined_condition].index

# Print the filtered indices
print(len(filtered_indices))

In [ ]:
114148/4

In [ ]:
5723499/(np.pi*(10**2) - np.pi*(2)**2)

In [ ]:
28537/18977.565783989998

In [ ]:
60493/2.25

In [ ]:
(85636/4)/18977.565783989998

In [ ]:
114148/85636

In [ ]:
xmin, xmax = -8, -6.5
ymin, ymax = -5.5, -4

# Define conditions
condition_x = (x_arr >= xmin) & (x_arr <= xmax)
condition_y = (y_arr >= ymin) & (y_arr <= ymax)

# Combine conditions
combined_condition = condition_x & condition_y

# Filter indices
filtered_indices = x_arr[combined_condition].index

# Print the filtered indices
print(len(filtered_indices))

In [ ]:
(1/np.tan(np.deg2rad(1.2))) * 0.85


In [ ]:
start_date = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
x_arr = df_good_data['impact_x_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)]
y_arr = df_good_data['impact_y_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)]
print(f"Number of points = {len(x_arr)}")
alpha_value = 0.005
fig, ax = plt.subplots(figsize=(12,12))
ax.set_aspect('equal')

ax.scatter(x_arr, y_arr, alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
#ax.set_title(f"Reconstructed impact points of observed {(upper_limit-low_limit):.0E} the muons on the LST mirror, transparency of dots = 1")

for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.3, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
#plt.savefig(f'/Users/vdk/muons2024/images//ImpactPoints_{len(x_arr)}muons_2023year_Alpha{alpha_value}.png', dpi=200, format='png', bbox_inches='tight')
# plt.xticks(np.arange(-11, 11, 0.5), fontsize = 7)  # Adjust the 0.5 to make it denser or less dense
# plt.yticks(np.arange(-11, 11, 0.5)) 
plt.grid(alpha=0.25)
#plt.show()

In [ ]:
plt.figure(figsize=(12,4))
z,x,c = plt.hist(df_good_data['impact_x_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)], bins = 100, alpha = 0.8, density=True)
plt.xlim(-12.5,12.5)
plt.grid(alpha = 0.7)

In [ ]:
start_date = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
x_arr = df_good_data['impact_x_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)]
y_arr = df_good_data['impact_y_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)]

alpha_value = 0.02
fig, ax = plt.subplots(figsize=(12,12))
ax.set_aspect('equal')

ax.scatter(x_arr, y_arr, alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
#ax.set_title(f"Reconstructed impact points of observed {(upper_limit-low_limit):.0E} the muons on the LST mirror, transparency of dots = 1")

for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.3, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
#plt.savefig(f'/Users/vdk/muons2024/images/ZenithImpactStudy/ImpactPoints_{(low_limit)}to{(upper_limit)}muons_Alpha{alpha_value}.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
df_good_sim_data.columns

### Simulated muons

In [ ]:
low_limit = 0
upper_limit = 2000000
alpha_value = 0.1
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
# circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
# ax.add_patch(circle)
# ax.set_aspect('equal')


ax.scatter(df_good_sim_data['impact_x_array'][~df_good_sim_data['parameters_at_limit'] & df_good_sim_data['is_valid']], df_good_sim_data['impact_y_array'][~df_good_sim_data['parameters_at_limit'] & df_good_sim_data['is_valid']], alpha = alpha_value, s = 1, marker = 'x')
#ax.scatter(without_problems['impact_x_array'], without_problems['impact_y_array'][:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of simulated {(upper_limit-low_limit):.0E} the muons on the LST mirror, z=40, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.3, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
#ax.legend(handles, labels)
#plt.savefig(f'/Users/vdk/muons2024/images/ZenithImpactStudy/SimImpactPoints_1e6muons_Alpha{alpha_value}.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
z,x,c = plt.hist(df_good_sim_data['impact_x_array'][~df_good_sim_data['parameters_at_limit'] & df_good_sim_data['is_valid']],  bins = 100, alpha = 0.8, density=True, label = 'Sim data')
#z,x,c = plt.hist(df_good_data['impact_y_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)], bins = 100, alpha = 0.8, density=True,label = 'Obs data')
plt.legend()
plt.xlim(-12.5,12.5)
plt.grid(alpha = 0.7)

In [ ]:
len(df_good_sim_data['impact_x_array'][~df_good_sim_data['parameters_at_limit'] & df_good_sim_data['is_valid']])

In [ ]:
df_good_data

In [ ]:
len(df_good_sim_data)

### Lets play now with transparency of a dots

#### Simulated data

In [ ]:
low_limit = 0
upper_limit = len(df_good_sim_data)
alpha_value = 1
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
# circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
# ax.add_patch(circle)
# # Set the aspect of the plot to equal to make sure the circle is not oval
# ax.set_aspect('equal')

#ax.scatter(df_good_sim_data['impact_x_array'][low_limit:upper_limit], df_good_sim_data['impact_y_array'][low_limit:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
#ax.scatter(df_good_sim_data['impact_x_array'][(df_good_sim_data['is_valid']) | (~df_good_sim_data['parameters_at_limit'])], df_good_sim_data['impact_y_array'][(df_good_sim_data['is_valid']) | (~df_good_sim_data['parameters_at_limit'])], alpha = alpha_value, s = 1, marker = 'x')
ax.scatter(df_good_sim_data['impact_x_array'][(~df_good_sim_data['parameters_at_limit'])], df_good_sim_data['impact_y_array'][(~df_good_sim_data['parameters_at_limit'])], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
#ax.set_title(f"Reconstructed impact points of the simulated {(upper_limit-low_limit):.0E} muons on the LST mirror, transparency of dots = {alpha_value}")#
ax.set_title(f"Reconstructed impact points of the simulated {(upper_limit-low_limit):.2E} muons on the LST mirror, transparency of dots = {alpha_value}")
ax.set_title(f"Reconstructed impact points of the simulated 1E+06 muons on the LST mirror, transparency of dots = {alpha_value}")
handles, labels = ax.get_legend_handles_labels()
for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.3, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)
#ax.legend(handles, labels)
#plt.savefig(f'/Users/vdk/muons2024/images/SimImpactPoints_NoParamLimit_zenith0_Alpha{alpha_value}.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
#z,x,c = plt.hist(df_good_sim_data['impact_x_array'][(df_good_sim_data['is_valid']) & (~df_good_sim_data['parameters_at_limit'])],  bins = 100, alpha = 0.8, density=True, label = 'Sim data')
z,x,c = plt.hist(df_good_sim_data['impact_x_array'][(~df_good_sim_data['parameters_at_limit'])],  bins = 100, alpha = 0.8, density=True, label = 'Sim data')
#z,x,c = plt.hist(df_good_data['impact_y_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)], bins = 100, alpha = 0.8, density=True,label = 'Obs data')
plt.legend()
plt.xlim(-12.5,12.5)
plt.grid(alpha = 0.7)

In [ ]:
plt.figure(figsize=(12,4))
#z,x,c = plt.hist(df_good_sim_data['impact_x_array'][(df_good_sim_data['is_valid']) & (~df_good_sim_data['parameters_at_limit'])],  bins = 100, alpha = 0.8, density=True, label = 'Sim data')
z,x,c = plt.hist(df_good_sim_data['impact_x_array'][(~df_good_sim_data['parameters_at_limit'])],  bins = 100, alpha = 0.8, density=True, label = 'Sim data')
#z,x,c = plt.hist(df_good_data['impact_y_array'][(df_good_data['event_time'] > start_date) & (df_good_data['event_time'] < end_date)], bins = 100, alpha = 0.8, density=True,label = 'Obs data')
plt.legend()
plt.xlim(-12.5,12.5)
plt.grid(alpha = 0.7)

### Percentage of unvalid events

In [ ]:
len(df_good_sim_data['impact_x_array'][(~df_good_sim_data['is_valid']) | (df_good_sim_data['parameters_at_limit'])])/len(df_good_sim_data['impact_x_array'])

### Optical efficiency in the invalid events

In [ ]:
np.mean(df_good_sim_data['muon_efficiency'][(df_good_sim_data['is_valid']) & (~df_good_sim_data['parameters_at_limit'])])
#np.mean(df_good_sim_data['muon_efficiency'])
len(df_good_sim_data['muon_efficiency'])
len(df_good_sim_data['muon_efficiency'][(df_good_sim_data['is_valid']) & (~df_good_sim_data['parameters_at_limit'])])
#len(df_good_sim_data['muon_efficiency'][(~df_good_sim_data['is_valid']) | (df_good_sim_data['parameters_at_limit'])])

#### Observaed data

In [ ]:
low_limit = 0
upper_limit = 1500000
alpha_value = 0.05
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
#circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
#ax.add_patch(circle)
for mirror_x, mirror_y in zip(first_column,second_column):
    circle = Circle((mirror_y*0.01, (mirror_x*0.01-0.269)), 0.75, color='k', fill=False, alpha = 0.5)  # Center at (0.5, 0.5) with radius 0.2
    ax.add_patch(circle)
# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_good_data['impact_x_array'][low_limit:upper_limit], df_good_data['impact_y_array'][low_limit:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of the observed {(upper_limit-low_limit):.0E} muons on the LST mirror, transparency of dots = {alpha_value}")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
df_to_check = df_good_data[low_limit:upper_limit]
df_to_check_sim = df_good_sim_data[low_limit:upper_limit]
stain_muons = df_to_check[
    (df_to_check['impact_y_array'] > 0) & 
    (df_to_check['impact_y_array'] < 10) & 
    (df_to_check['impact_x_array'] > -2) & 
    (df_to_check['impact_x_array'] < 2)
]
regular_muons = df_to_check[
    (df_to_check['impact_y_array'] > -2) & 
    (df_to_check['impact_y_array'] < 2) & 
    (df_to_check['impact_x_array'] > 0) & 
    (df_to_check['impact_x_array'] < 10)
]
sim_muons = df_to_check_sim[
    (df_to_check_sim['impact_y_array'] > -2) & 
    (df_to_check_sim['impact_y_array'] < 2) & 
    (df_to_check_sim['impact_x_array'] > 0) & 
    (df_to_check_sim['impact_x_array'] < 10)
]
print(f"Number of muons in the more dense region = {len(stain_muons)}")
print(f"Number of muons in the region of lower density {len(regular_muons)}")
print(f"Number of muons in the same region in simulations {len(sim_muons)}")

In [ ]:
min(df_good_sim_data['impact_parameter'])

In [ ]:


low_limit = 1500000
upper_limit = 3000000
alpha_value = 0.01
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(9,9))
# Create a circle patch
# circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2
# ax.add_patch(circle)



# def hexagon(center_x, center_y, radius):
#     """ Generate the vertices of a hexagon given center and radius """
#     angle = np.linspace(0, 2 * np.pi, 7)
#     x_hexagon = center_x + radius * np.sin(angle)
#     y_hexagon = center_y + radius * np.cos(angle)
#     return x_hexagon, y_hexagon
# x_hexagons=[]
# y_hexagons=[]
# # Plotting hexagons
# for center_x, center_y in zip(first_column, second_column):
#     x_hexagon, y_hexagon = hexagon(center_x*0.01, center_y*0.01, 0.75)
#     print(center_x, x_hexagon)
#     print(center_y, y_hexagon)
#     hex_patch = patches.Polygon(np.column_stack([x_hexagon, y_hexagon]), closed=True, edgecolor='r', facecolor='none')
#     ax.add_patch(hex_patch)

for center_x, center_y in zip(first_column, second_column):
    hex = RegularPolygon((center_y*0.01, center_x*0.01-0.269), numVertices=6, radius=0.85, alpha=0.7, edgecolor='k', fill=False)
    ax.add_patch(hex)
plt.autoscale(enable = True)

# for mirror_x, mirror_y in zip(first_column,second_column):
#     circle = Circle((mirror_y*0.01, (mirror_x*0.01-0.269)), 0.75, color='k', fill=False, alpha = 0.5)  # Center at (0.5, 0.5) with radius 0.2
#     ax.add_patch(circle)
# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_good_data['impact_x_array'][low_limit:upper_limit], df_good_data['impact_y_array'][low_limit:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of the observed {(upper_limit-low_limit):.0E} muons on the LST mirror, transparency of dots = {alpha_value}")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
np.column_stack([x_hexagon, y_hexagon])

In [ ]:
# So, how much impact points we expect in one square element
N = 1500000
S1 = np.pi * (max(df_good_data['impact_parameter']))**2
S2 = np.pi * (min(df_good_data['impact_parameter']))**2
S = S1 - S2
print(f"We expect to have '{round(N/S,1)}' muons per area unit")

In [ ]:
# So, how much impact points we expect in one square element
N = 1500000
S1 = np.pi * (max(df_good_data['impact_parameter']))**2
S2 = np.pi * (min(df_good_data['impact_parameter']))**2
S = S1 - S2
df_to_check = df_good_data[low_limit:upper_limit]
stain_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] > 1) & 
    (df_to_check['impact_x_array'] < 4)
]
regular_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] < -3) & 
    (df_to_check['impact_x_array'] > -6)
]
print(f"Number of muons in the more dense region = {len(stain_muons)/(3*3)}")
print(f"Number of muons in the region of lower density {len(regular_muons)/(3*3)}")
print(f"We expect to have '{round(N/S,1)}' muons per area unit")


In [ ]:
stain_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] > 1) & 
    (df_to_check['impact_x_array'] < 4)
]

In [ ]:
low_limit = 3000000
upper_limit = 4500000
alpha_value = 0.05
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2

# Add the circle to the Axes
ax.add_patch(circle)

# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_good_data['impact_x_array'][low_limit:upper_limit], df_good_data['impact_y_array'][low_limit:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title("Reconstructed impact points of the observed muons on the LST mirror, transparency of dots = 1")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
plt.grid(alpha=0.15)
plt.show()

In [ ]:
df_to_check = df_good_data[5000000:6000000]
df_to_check[(df_to_check['impact_y_array'] > -10) & (df_to_check['impact_y_array'] < -7) & (df_to_check['impact_x_array'] > 0) & (df_to_check['impact_x_array'] < 4)]

In [ ]:
df_to_check[(df_to_check['impact_y_array'] > -10) & (df_to_check['impact_y_array'] < -7) & (df_to_check['impact_x_array'] < 0) & (df_to_check['impact_x_array'] > -4)]

In [ ]:
df_to_check = df_good_data[5000000:6000000]
df_to_check[(df_to_check['impact_y_array'] > -10) & (df_to_check['impact_y_array'] < -7) & (df_to_check['impact_x_array'] > 0) & (df_to_check['impact_x_array'] < 4)]

### Sorted by event time

In [ ]:
df_sorted = df_good_data.sort_values(by='event_time', ascending=True)

In [ ]:
low_limit = 1700000
upper_limit = 1900000
alpha_value = 0.2
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
# circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')  # Center at (0.5, 0.5) with radius 0.2

# # Add the circle to the Axes
# ax.add_patch(circle)
for mirror_x, mirror_y in zip(first_column,second_column):
    circle = Circle((mirror_y*0.01, (mirror_x*0.01-0.269)), 0.75, color='k', fill=False, alpha = 0.5)  # Center at (0.5, 0.5) with radius 0.2
    ax.add_patch(circle)

# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_sorted['impact_x_array'][low_limit:upper_limit], df_sorted['impact_y_array'][low_limit:upper_limit], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of the observed {(upper_limit-low_limit):.0E} muons on the LST mirror, transparency of dots = {alpha_value}")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
print(f"Start date {datetime.fromtimestamp(min(df_sorted['event_time'][low_limit:upper_limit])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
print(f"End date {datetime.fromtimestamp(max(df_sorted['event_time'][low_limit:upper_limit])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
#plt.grid(alpha=0.15)
plt.grid(alpha=1)
plt.show()

In [ ]:
low_limit = 1700000
upper_limit = 1900000
alpha_value = 0.1
# Create a new figure and an axes instance
fig, ax = plt.subplots(figsize=(12,12))
# Create a circle patch
circle = Circle((0, 0), 11.5, color='red', fill=False, label = 'LST1 full mirror')

# Add the circle to the Axes
ax.add_patch(circle)

# Set the aspect of the plot to equal to make sure the circle is not oval
ax.set_aspect('equal')

ax.scatter(df_to_check_sim['impact_x_array'], df_to_check_sim['impact_y_array'], alpha = alpha_value, s = 1, marker = 'x')
ax.set_xlabel("Impact parameter X [m]")
ax.set_ylabel("Impact parameter Y [m]")
ax.set_title(f"Reconstructed impact points of the simulated {(upper_limit-low_limit):.0E} muons on the LST mirror, transparency of dots = {alpha_value}")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles, labels)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')
print(f"Start date {datetime.fromtimestamp(min(df_sorted['event_time'][low_limit:upper_limit])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
print(f"End date {datetime.fromtimestamp(max(df_sorted['event_time'][low_limit:upper_limit])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
plt.grid(alpha=0.15)
plt.show()

In [ ]:
df_to_check = df_sorted[low_limit:upper_limit]
df_to_check_sim = df_good_sim_data[1200000:1400000]
stain_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] > 1) & 
    (df_to_check['impact_x_array'] < 4)
]
regular_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] < -3) & 
    (df_to_check['impact_x_array'] > -6)
]
sim_muons = df_to_check_sim[
    (df_to_check_sim['impact_y_array'] > -9) & 
    (df_to_check_sim['impact_y_array'] < -6) & 
    (df_to_check_sim['impact_x_array'] < -3) & 
    (df_to_check_sim['impact_x_array'] > -6)
]
print(f"Number of muons in the more dense region = {len(stain_muons)}")
print(f"Number of muons in the region of lower density {len(regular_muons)}")
print(f"Number of muons in the same region in simulations {len(sim_muons)}")

In [ ]:
df_to_check = df_sorted[low_limit:upper_limit]
df_to_check_sim = df_good_sim_data[1200000:1400000]
stain_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] > 1) & 
    (df_to_check['impact_x_array'] < 4)
]
regular_muons = df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] > 3) & 
    (df_to_check['impact_x_array'] < 5)
]
sim_muons = df_to_check_sim[
    (df_to_check_sim['impact_y_array'] > -9) & 
    (df_to_check_sim['impact_y_array'] < -6) & 
    (df_to_check_sim['impact_x_array'] > 1) & 
    (df_to_check_sim['impact_x_array'] < 4)
]
print(f"Number of muons in the more dense region = {len(stain_muons)}")
print(f"Number of muons in the region of lower density {len(regular_muons)}")
print(f"Number of muons in the same region in simulations {len(sim_muons)}")

In [ ]:
df_to_check = df_sorted[low_limit:upper_limit]
df_to_check[
    (df_to_check['impact_y_array'] > -9) & 
    (df_to_check['impact_y_array'] < -6) & 
    (df_to_check['impact_x_array'] > 3) & 
    (df_to_check['impact_x_array'] < 5)
]

### Heatmap

In [ ]:
# Assuming muon_centers and mirrors_centers are numpy arrays of shape (N, 2) for 2D coordinates
# Replace with your actual data


# Initialize the mirror_numbers array


# Vectorize distance calculations
for muon in muon_centers:
    # Calculate distances from the current muon to all mirrors
    distances = np.linalg.norm(mirrors_centers - muon, axis=2)
    
    # Find the index of the closest mirror
    closest_mirror_index = np.argmin(distances)
    
    # Increment the count for the closest mirror
    mirror_numbers[closest_mirror_index] += 1

print(mirror_numbers)


In [ ]:
muon_centers_full = np.vstack((df_good_sim_data['impact_x_array'][~df_good_sim_data['parameters_at_limit']], df_good_sim_data['impact_y_array'][~df_good_sim_data['parameters_at_limit']]))
mirrors_centers_full = np.vstack((first_column, second_column))

# Assuming muon_centers and mirrors_centers are numpy arrays of shape (N, 2) for 2D coordinates
# Replace with your actual data
muon_centers = np.random.rand(198, 2)  # Example data
mirrors_centers = np.random.rand(15000, 2)  # Example data

In [ ]:
mirror_numbers = np.zeros(len(mirrors_centers), dtype=int)
muon_centers = np.vstack((df_good_sim_data['impact_x_array'][~df_good_sim_data['parameters_at_limit'] & df_good_sim_data['is_valid']], df_good_sim_data['impact_y_array'][~df_good_sim_data['parameters_at_limit'] & df_good_sim_data['is_valid']])).T
#muon_centers = np.vstack((df_good_sim_data['impact_x_array'][df_good_sim_data['is_valid']], df_good_sim_data['impact_y_array'][df_good_sim_data['is_valid']])).T
#muon_centers = np.vstack((df_good_sim_data['impact_x_array'][~df_good_sim_data['is_valid']], df_good_sim_data['impact_y_array'][~df_good_sim_data['is_valid']])).T
#muon_centers = np.vstack((df_good_sim_data['impact_x_array'][~df_good_sim_data['parameters_at_limit']], df_good_sim_data['impact_y_array'][~df_good_sim_data['parameters_at_limit']])).T

mirrors_centers = np.vstack((np.array(second_column)*0.01, np.array(first_column)*0.01)).T

print(muon_centers.shape)
print(mirrors_centers.shape)
# Initialize the mirror_numbers array
mirror_numbers = np.zeros(len(mirrors_centers), dtype=int)

# Vectorize distance calculations
for muon in muon_centers:
    # Calculate distances from the current muon to all mirrors
    distances = np.linalg.norm(mirrors_centers - muon, axis=1)
    
    # Find the index of the closest mirror
    closest_mirror_index = np.argmin(distances)
    
    # Increment the count for the closest mirror
    mirror_numbers[closest_mirror_index] += 1

#print(mirror_numbers)
# Separate the coordinates into x and y lists
x, y = zip(*mirrors_centers)

# Create the scatter plot
plt.figure(figsize=(12, 9))
scatter = plt.scatter(x, y, c=mirror_numbers, cmap='viridis', s=1100, edgecolor='k', marker = 'h')

# Add a colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Number of Events')

# Add labels and title
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Heatmap of Events around Coordinates')

# Show the plot
plt.show()


In [ ]:
mirror_numbers = np.zeros(len(mirrors_centers), dtype=int)
muon_centers = np.vstack((df_good_data['impact_x_array'], df_good_data['impact_y_array'])).T
mirrors_centers = np.vstack((np.array(second_column)*0.01, np.array(first_column)*0.01)).T

print(muon_centers.shape)
print(mirrors_centers.shape)
# Initialize the mirror_numbers array
mirror_numbers = np.zeros(len(mirrors_centers), dtype=int)

# Vectorize distance calculations
for muon in muon_centers:
    # Calculate distances from the current muon to all mirrors
    distances = np.linalg.norm(mirrors_centers - muon, axis=1)
    
    # Find the index of the closest mirror
    closest_mirror_index = np.argmin(distances)
    
    # Increment the count for the closest mirror
    mirror_numbers[closest_mirror_index] += 1

print(mirror_numbers)
# Separate the coordinates into x and y lists


In [ ]:
x, y = zip(*mirrors_centers)

# Create the scatter plot
plt.figure(figsize=(12, 9.5))
scatter = plt.scatter(x, y, c=mirror_numbers, cmap='viridis', s=1400, edgecolor='k', marker = 'h')

# Add a colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Number of Events')

# Add labels and title
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Heatmap of Events around Coordinates')

# Show the plot
plt.show()